In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    libcblas-3.8.0             |      11_openblas          10 KB  conda-forge
    libblas-3.8.0              |      11_openblas          10 KB  conda-forge
    blas-2.11                  |         openblas          10 KB  conda-forge
    ------------------------------------------------------------
       

In [2]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.4            |           py36_0          58 KB  conda-forge
    beautifulsoup4-4.8.2       |           py36_0         157 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         215 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.4-py36_0 conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0             --> 4.8.2-py36_0 conda-forge


soupsieve-1.9.4      

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url)

In [5]:
My_table = soup.find('table', {'class':'wikitable sortable'})


In [6]:
links = My_table.findAll('td')

In [7]:
Postal1=[]
Borough1=[]
Neighborhood1=[]
i = 0
for name in links:
    name = str(name)
    name =name.replace('<td>', '').replace('</td>','')
    if i == 0:
        Postal1.append(name)
        i = 1
        continue
    if i == 1:
        Borough1.append(name)
        i = 2
        continue
    if i == 2:
        Neighborhood1.append(name)
        i = 0
        continue

In [8]:
Postal = Postal1
Borough = Borough1
Neighborhood = Neighborhood1

In [9]:
import re

In [10]:
# Extract the value of beautiful soup. Omit the value of Not assigned in the Borough
for i in range (0,len(Borough)):
    if Borough[i]!='Not assigned':
        r = re.compile('title=".*"')
        temp = r.findall(Borough[i])
        if len(temp)!=0:
            temp = temp[0][7:len(temp[0])-1]
            Borough[i] = temp

In [11]:
# Extract the value of beautiful soup. Omit the value of Not assigned in the Borough
for i in range (0,len(Neighborhood)):
    if 'Not assigned' in Neighborhood[i]:
        Neighborhood[i] = Borough[i]
    else:
        r = re.compile('title=".*"')
        temp = r.findall(Neighborhood[i])
        if len(temp)!=0:
            temp = str(temp[0][7:len(temp[0])-1])
            Neighborhood[i] = temp

In [12]:
for i in range(0, len(Neighborhood)):
    if '\n' in Neighborhood[i]:
        Neighborhood[i]=Neighborhood[i].replace('\n','')

In [13]:
import pandas as pd
df = pd.DataFrame()
df['Postal'] = Postal
df['Borough'] = Borough
df['Neighborhood'] = Neighborhood

In [14]:
df.head()

,Postal,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park


In [15]:
# Remove the value of 'not assigned' in Borough
df=df[df['Borough'] != 'Not assigned']

In [16]:
df=df.sort_values(['Postal'])
df=df.reset_index(drop=True)

In [17]:
# Combine the Borough and Neighborhood is the postal code is the same
aggregation_functions = {'Postal':'first','Borough':'first','Neighborhood':'sum'}
df_new = df.groupby(df['Postal']).aggregate(aggregation_functions)

In [18]:
df_new= df_new.reset_index(drop=True)

In [19]:
# print the shape of the final dataframe

df_new.shape

(103, 3)

In [20]:
df_new

,Postal,Borough,Neighborhood
0,M1B,"Scarborough, Toronto","Rouge, TorontoMalvern, Toronto"
1,M1C,"Scarborough, Toronto","Port Union, TorontoRouge HillHighland Creek (T..."
2,M1E,"Scarborough, Toronto","GuildwoodMorningside, TorontoWest Hill, Toronto"
3,M1G,"Scarborough, Toronto","Woburn, Toronto"
4,M1H,"Scarborough, Toronto",Cedarbrae
5,M1J,"Scarborough, Toronto",Scarborough Village
6,M1K,"Scarborough, Toronto","East Birchmount ParkIonviewKennedy Park, Toronto"
7,M1L,"Scarborough, Toronto","Golden Mile, TorontoOakridge, TorontoClairlea"
8,M1M,"Scarborough, Toronto","CliffcrestScarborough Village WestCliffside, T..."
9,M1N,"Scarborough, Toronto",Cliffside WestBirch Cliff


In [21]:
df_second = pd.read_csv("Geospatial_Coordinates.csv")
df_new = df_new.join(df_second)
# drop the duplicated postal code
df_new.drop('Postal Code', axis = 1, inplace = True)
df_new

,Postal,Borough,Neighborhood,Latitude,Longitude
0,M1B,"Scarborough, Toronto","Rouge, TorontoMalvern, Toronto",43.806686,-79.194353
1,M1C,"Scarborough, Toronto","Port Union, TorontoRouge HillHighland Creek (T...",43.784535,-79.160497
2,M1E,"Scarborough, Toronto","GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711
3,M1G,"Scarborough, Toronto","Woburn, Toronto",43.770992,-79.216917
4,M1H,"Scarborough, Toronto",Cedarbrae,43.773136,-79.239476
5,M1J,"Scarborough, Toronto",Scarborough Village,43.744734,-79.239476
6,M1K,"Scarborough, Toronto","East Birchmount ParkIonviewKennedy Park, Toronto",43.727929,-79.262029
7,M1L,"Scarborough, Toronto","Golden Mile, TorontoOakridge, TorontoClairlea",43.711112,-79.284577
8,M1M,"Scarborough, Toronto","CliffcrestScarborough Village WestCliffside, T...",43.716316,-79.239476
9,M1N,"Scarborough, Toronto",Cliffside WestBirch Cliff,43.692657,-79.264848


## Explore Neighborhoods in Canada

### Define the getNearbyVenues function and configure the foursquare api settings

In [22]:
CLIENT_ID = '4MW03F14E1XNATNJVQCBZMKQAQXB5AGD2DCDNTV5WEINUUKM' # your Foursquare ID
CLIENT_SECRET = '2WOHKIJOJAB2JJPJVRBHCMKOSYMZP4KFNRYPCJKMEGDLENA5'
VERSION = '20180605' # Foursquare API version
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Canada_venues = getNearbyVenues(names=df_new['Neighborhood'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )

In [ ]:
Canada_venues.head()

In [ ]:
Canada_venues.groupby('Neighborhood').count()

In [26]:
print('There are {} uniques categories.'.format(len(Canada_venues['Venue Category'].unique())))

There are 263 uniques categories.


In [27]:
Canada_onehot = pd.get_dummies(Canada_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Canada_onehot['Neighborhood'] = Canada_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Canada_onehot.columns[-1]] + list(Canada_onehot.columns[:-1])
Canada_onehot = Canada_onehot[fixed_columns]

Canada_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [28]:
Canada_grouped = Canada_onehot.groupby('Neighborhood').mean().reset_index()

In [29]:
Canada_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,AdelaideRichmondKing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.000000,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.01,0.0,0.000000,0.0,0.01,0.03,0.01,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0

#### Cluster Neighbourhood


Run *k*-means to cluster the neighborhood into 5 clusters.

In [30]:
kclusters = 5

Canada_grouped_clustering = Canada_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Canada_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 3, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 0, 1], dtype=int32)

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Canada_grouped['Neighborhood']

for ind in np.arange(Canada_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Canada_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AdelaideRichmondKing,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Restaurant,Asian Restaurant,Burger Joint,Cosmetics Shop,Bakery
1,"Agincourt, Toronto",Latin American Restaurant,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store
2,Albion GardensBeaumond HeightsHumbergateMount ...,Grocery Store,Pizza Place,Beer Store,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Liquor Store,Coffee Shop,Pharmacy,Colombian Restaurant
3,"Bathurst ManorWilson Heights, TorontoDownsview...",Coffee Shop,Fried Chicken Joint,Sandwich Place,Diner,Bridal Shop,Restaurant,Deli / Bodega,Bank,Ice Cream Shop,Supermarket
4,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [48]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Canada_merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Canada_merged = Canada_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Canada_merged.head() # check the last columns!

,Postal,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Scarborough, Toronto","Rouge, TorontoMalvern, Toronto",43.806686,-79.194353,4.0,Fast Food Restaurant,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
1,M1C,"Scarborough, Toronto","Port Union, TorontoRouge HillHighland Creek (T...",43.784535,-79.160497,2.0,Bar,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
2,M1E,"Scarborough, Toronto","GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,0.0,Rental Car Location,Mexican Restaurant,Spa,Breakfast Spot,Medical Center,Intersection,Electronics Store,Pizza Place,Dessert Shop,Dumpling Restaurant
3,M1G,"Scarborough, Toronto","Woburn, Toronto",43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
4,M1H,"Scarborough, Toronto",Cedarbrae,43.773136,-79.239476,0.0,Gas Station,Thai Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Caribbean Restaurant,Bakery,Hakka Restaurant,Donut Shop,Doner Restaurant


In [51]:
Canada_merged=Canada_merged[Canada_merged['Cluster Labels'].isnull() == False]

In [52]:
latitude = 43.806
longitude = -79.194
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Canada_merged['Latitude'], Canada_merged['Longitude'], Canada_merged['Neighborhood'], Canada_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters